# **What is Churn Model?**

*A churn model is a mathematical representation of how churn impacts your business. Churn calculations are built on existing data (the number of customers who left your service during a given time period). A predictive churn model extrapolates on this data to show future potential churn rates.*

*In its simplest form, churn rate is calculated by dividing the number of customer cancellations within a time period by the number of active customers at the start of that period. Very valuable insights can be gathered from this simple analysis — for example, the overall churn rate can provide a benchmark against which to measure the impact of a model. And knowing how churn rate varies by time of the week or month, product line, or customer cohort can help inform simple customer segments for targeting as well.*

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Importing Libraries**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt

# **Importing Dataset**

In [ ]:
dataset = pd.read_csv('/kaggle/input/churn-modelling/Churn_Modelling.csv')

In [ ]:
dataset

**Displaying info of dataset.**

In [ ]:
dataset.info()

**Check if there are any NULL values present.**

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.describe()

**Check correlation in your dataset.**

In [ ]:
dataset.corr()

# **Data Visualization**

**Heat Map Correlation**

In [ ]:
# Compute the correlation matrix
corr = dataset.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})

**Plotting with Age**

In [ ]:
fig = px.violin(dataset, y= 'Age')
fig.show()

**Age vs Gender**

In [ ]:
fig = px.violin(dataset, x="Age", y="Gender", orientation= 'h')
fig.show()

**Age vs Geography**

In [ ]:
fig = px.violin(dataset, x="Age", y="Geography", orientation= 'h')
fig.show()

**Plotting with Geography**

**Geography vs Estimated Salary**

In [ ]:
fig = px.violin(dataset, x="Geography", y="EstimatedSalary", color = 'Gender', violinmode='overlay', hover_data=dataset.columns)
fig.show()

**Geography vs Balance**

In [ ]:
fig = px.violin(dataset, x="Geography", y="Balance", color = 'Gender', violinmode='overlay', hover_data=dataset.columns)
fig.show()

**Distribution Plot**

In [ ]:
fig = plt.figure(figsize=(7,7))
sns.distplot(dataset.EstimatedSalary, color="green", label="Estimated Salary", kde= True)
plt.legend()

In [ ]:
fig = plt.figure(figsize=(7,7))
sns.distplot(dataset.Balance, color="blue", label="Estimated Salary", kde= True)
plt.legend()

In [ ]:
fig = plt.figure(figsize=(7,7))
sns.distplot(dataset.CreditScore, color="red", label="Estimated Salary", kde= True)
plt.legend()

**Pair Plot**

In [ ]:
sns.pairplot(dataset)

# **Data Preprocessing**

In [ ]:
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
x

In [ ]:
y

# **Encoding Categorical Data**

**Label Encoding the "Gender" column**

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])

In [ ]:
x

**One Hot Encoding the "Geography" column**

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
x = np.array(ct.fit_transform(x))

In [ ]:
x

# **Splitting the dataset into the Training set and Test set**

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state= 0)

# **Feature Scaling**

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# **Building the ANN**

**Importing Regularizers to add a penalty for weight size to the loss function and avoiding overfitting**

In [ ]:
from keras.regularizers import l2

**Initializing the ANN**

In [ ]:
ann = tf.keras.models.Sequential()

**Adding the input layer and the first hidden layer**

In [ ]:
ann.add(tf.keras.layers.Dense(units= 8, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu'))

**Adding first Dropout layer**

In [ ]:
tf.keras.layers.Dropout(0.6)

**Adding the second hidden layer**

In [ ]:
ann.add(tf.keras.layers.Dense(units= 8, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu'))

**Adding second Dropout layer**

In [ ]:
tf.keras.layers.Dropout(0.6)

**Adding the output layer**

In [ ]:
ann.add(tf.keras.layers.Dense(units= 1, activation='sigmoid'))

# **Training the ANN**

In [ ]:
ann.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])

In [ ]:
ann_history = ann.fit(x_train, y_train, batch_size= 32, epochs= 100, validation_split= 0.3)

# **Visualizing Training and Validation Loss**

In [ ]:
loss_train = ann_history.history['loss']
loss_val = ann_history.history['val_loss']
epochs = range(1,101)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# **Visualizing Training and Validation Accuracy**

In [ ]:
loss_train = ann_history.history['accuracy']
loss_val = ann_history.history['val_accuracy']
epochs = range(1,101)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# **Visualizing Confusion Matrix**

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay

In [ ]:
# Predicting the Test set results
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

# Calculate the Accuracy
accuracy = accuracy_score(y_pred,y_test)

In [ ]:
cm

In [ ]:
accuracy

In [ ]:
#Predicting on new data
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

In [ ]:
cmd = ConfusionMatrixDisplay(cm, display_labels=['Stay','Leave'])
cmd.plot()

# **Evluating the ANN (Cross Validation)**

**Wrapping k-fold cross validation into keras model**

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
# Builing the function
def ann_classifier():
    ann = tf.keras.models.Sequential()
    ann.add(tf.keras.layers.Dense(units= 8, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu'))
    ann.add(tf.keras.layers.Dense(units= 8, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu'))
    tf.keras.layers.Dropout(0.6)
    ann.add(tf.keras.layers.Dense(units= 1, activation='sigmoid'))
    ann.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])
    return ann

**Performing the Cross Validation**

In [ ]:
# Passing values to KerasClassifier 
ann = KerasClassifier(build_fn = ann_classifier, batch_size = 32, epochs = 100)

In [ ]:
# We are using 5 fold cross validation here
accuracies = cross_val_score(estimator = ann, X = x_train, y = y_train, cv = 5)

In [ ]:
# Checking the mean and standard deviation of the accuracies obtained
mean = accuracies.mean()
std_deviation = accuracies.std()
print("Accuracy: {:.2f} %".format(mean*100))
print("Standard Deviation: {:.2f} %".format(std_deviation*100))

# **Tuning the ANN**

**We use the Grid Search method for this task**

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Builing the function
def ann_classifier(optimizer = 'adam'):
    ann = tf.keras.models.Sequential()
    ann.add(tf.keras.layers.Dense(units= 8, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu'))
    ann.add(tf.keras.layers.Dense(units= 8, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu'))
    tf.keras.layers.Dropout(0.6)
    ann.add(tf.keras.layers.Dense(units= 1, activation='sigmoid'))
    ann.compile(optimizer= optimizer, loss= 'binary_crossentropy', metrics= ['accuracy'])
    return ann

In [ ]:
# Passing values to KerasClassifier 
ann = KerasClassifier(build_fn = ann_classifier, batch_size = 32, epochs = 100)

In [ ]:
# Using Grid Search CV to getting the best parameters
parameters = {'batch_size': [25, 32],
             'epochs': [100, 150],
             'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = ann, param_grid = parameters, scoring = 'accuracy', cv = 5, n_jobs = -1)

grid_search.fit(x_train, y_train)

In [ ]:
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

In [ ]:
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

# **Running the ANN again based on parameters obtained above**

In [ ]:
# defining the layers
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units= 8, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu'))
ann.add(tf.keras.layers.Dense(units= 8, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu'))
tf.keras.layers.Dropout(0.6)
ann.add(tf.keras.layers.Dense(units= 1, activation='sigmoid'))
ann.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])
ann.fit(x_train, y_train, batch_size= 32, epochs= 150)

In [ ]:
# Predicting the Test set results
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

# Calculate the Accuracy
accuracy = accuracy_score(y_pred,y_test)

In [ ]:
print('Confusion Matrix after tuning the ANN:\n', cm)

In [ ]:
print('Accuracy after tuning the ANN:', (accuracy)*100,'%')

# **Conclusion**

***Therefore, after tuning the ANN model, accuracy reaches 86.3% and increases by only 1%.***